In [1]:
import streamlit as st
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

# Initialize the Spotipy client with your Spotify API credentials
client_id = 'b7e3715f645d480b93687e744c4287ab'
client_secret = '0bb4c5c2bfef4248933cbf03478756ae'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_playlist_info(playlist_url, playlist_no):
    """Fetch general information and audio features for a Spotify playlist."""
    print("Executing.")

    playlist_id = playlist_url.split("/")[-1].split("?")[0] # Extract only the ID from the URL provided
    playlist = sp.playlist(playlist_id) # Fetch the entire playlist from spotify by its ID. If this fails, the Exception will be caught during the Streamlit runtime.

    tracks = playlist["tracks"]
    songs = tracks['items']

    while tracks['next']:
        tracks = sp.next(tracks)
        print("Tracks Next.")
        [songs.append(item) for item in tracks['items']]


    song_ids = [songs[i]['track']['id'] for i in range(0, len(songs))]
    print("Got IDs.")

    audio_features = []
    for i in range(0, len(song_ids), 50):
        print("Getting Audio Features")
        audio_features = sp.audio_features(song_ids[i:i + 50])
        for track in audio_features:
            if track is not None:
                audio_features.append(track)
                audio_features[len(audio_features) - 1]['playlist-no'] = playlist_no

    song_details = []


    # Loop through each song to fetch its details and audio features
    for song in songs:
        print("Looping!")
        track = song['track']
        track_id = track['id']
        title = track['name']
        artist = track['artists'][0]['name']  # Assuming the first artist is the primary
        album = track['album']['name']
        duration = track['duration_ms']
        created_date = track['album']['release_date']  # Release date of the album

        # Fetch the audio features for the track
        audio_feature = sp.audio_features(track_id)[0]  # Fetch the audio features
        
        # Combine the track details with its audio features (excluding redundant 'id')
        song_info = {
            'Track ID': track_id,
            'Title': title,
            'Artist': artist,
            'Album': album,
            'Duration': duration,
            'Created Date': created_date,
        }
        
        # Add the audio features to the song info, if available
        for key, value in audio_features.items():
            if key != 'id':  # Avoid duplicating the track ID
                    song_info[key] = value
        
        song_details.append(song_info)

        df = pd.DataFrame(song_details)

        return df


    
    info = {
        'Title': playlist['name'],
        'Created By': playlist['owner']['display_name'],
        'Number of Songs': len(songs),
    }
    
    
    return df, info


In [2]:
get_playlist_info('https://open.spotify.com/playlist/4RofcSdvlSn8IuNdzXXWOf?si=14cf348d7105417d',1)

Executing.
Got IDs.
Getting Audio Features


MemoryError: 